In [14]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
import pickle 

In [15]:
df= pd.read_csv('Data.csv')
df = df.sort_values('Location')

In [16]:
df.head()

,Location,Social Category,PrimaryGirls,PrimaryBoys,PrimaryOverall,UpperGirls,UpperBoys,UpperOverall,SecondGirls,SecondBoys,SecondOverall
0,AHMEDABAD,General,0.00,0.00,0.00,0.00,0.00,0.00,8.02,12.34,10.54
633,AHMEDABAD,OBC,18.12,18.26,18.19,26.75,19.81,22.98,35.95,39.83,38.35
632,AHMEDABAD,General,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.50,0.42
764,AHMEDABAD,General,9.23,12.54,11.02,12.00,12.73,12.40,12.77,21.93,18.05
765,AHMEDABAD,OBC,0.00,0.00,0.00,0.43,0.00,0.00,16.72,24.65,21.60


In [29]:
def find(overall,girl,boy):
    x = df[[overall]]
    y = df[[girl,boy]]
    x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=12,test_size=0.25)
    lr = LinearRegression()
    lr.fit(x_train,y_train)
    prediction = lr.predict(x_test)
    lr.score(x_test,y_test)
    print("R2 score : ",metrics.r2_score(y_test,prediction))
    print("MAE : ", metrics.mean_absolute_error(y_test,prediction))
    
    pickle.dump(lr, open('model.pkl', 'wb'))
    pickled_model = pickle.load(open('model.pkl', 'rb'))
    pickled_model.predict(x_test)

In [30]:
find('PrimaryOverall','PrimaryGirls', 'PrimaryBoys')

R2 score :  0.9841161708356758
MAE :  0.605099392407009
LinearRegression()


In [31]:
find('UpperOverall','UpperGirls', 'UpperBoys')

R2 score :  0.9376972322914263
MAE :  1.4950819340270072
LinearRegression()


In [32]:
find('SecondOverall','SecondGirls', 'SecondBoys')

R2 score :  0.8959536041747074
MAE :  2.768119944933511
LinearRegression()
